<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12. Special Methods for Sequences

Implementation of `Vector` class with following properties:
 1. Basic sequence protocol: `__len__` and `__getitem__`
 2. Safe representation of instances with many items
 3. Proper slicing support, producing new Vector instances
 4. Aggregate hashing, taking  into account every contained element value
 5. Custom formatting language extension

# Vector Take #1: Vector2d Compatible

best practice for a sequence constructor is to take the data as iterable argument in the constructor.

In [1]:
# vector_v1.py
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [3]:
tuple(Vector([3.1, 4.2]))

(3.1, 4.2)

In [4]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## Protocols and Duck Typing

You don't need to inherit from any special class to create a fully functional sequence type in Python; you just need to implement the methods that fulfill the sequence protocol

In [6]:
# Code from Ex1-1
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]


The above `FrenchDeck` class is a sequence even if it subclasses `object`. We say it is a sequence because it behaves like one, and that is what matters.

# Vector Take #2: A Sliceable Sequence

In [7]:
# vector_v2.py
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [8]:
v1 = Vector([3, 4, 5])

In [9]:
len(v1)

3

In [10]:
v1[0], v1[-1]

(3.0, 5.0)

In [11]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

## how slicing works

In [12]:
class MySeq:
  def __getitem__(self, index):
    return index

In [13]:
s = MySeq()

In [14]:
s[1]

1

In [15]:
s[1:4]

slice(1, 4, None)

In [16]:
s[1:4:2]

slice(1, 4, 2)

In [17]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [18]:
s[1:4:2, 7:9] # tuple may even hold several slice objects

(slice(1, 4, 2), slice(7, 9, None))

In [19]:
slice

slice

In [20]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [23]:
'ABCDE'[:10:2] == 'ABCDE'[0:5:2]

True

In [24]:
slice(None, 10, 2).indices(5) # indices exposes the tricky logic that's implemented in the built-in seq
                              # to gracefully handle missing or negative indices

(0, 5, 2)

In [25]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In our vector code, we'll not need the `slice.indices()` method because when we get a slice argument we'll delegate its handling t the _components array.

In [30]:
# vector_v2.py
from array import array
import operator
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [31]:
v7 = Vector(range(7))

In [32]:
v7[-1]

6.0

In [33]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [34]:
v7[-1:]

Vector([6.0])

In [35]:
v7[1, 2] # Vector does not support multidimensional indexing

TypeError: 'tuple' object cannot be interpreted as an integer